#### Regression modeling of house prices in King County, Washington sdf

In [387]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler

This notebook details construction of our regression model for predicting housing prices in King County, Washington.  First we read in the data set.

In [388]:
king = pd.read_csv('kc_house_data.csv')

In [389]:
king.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


This step trims down the dataset to the most relevant columns.  

In [390]:
king = king[['price','bedrooms','sqft_living','grade','sqft_lot']]
king.head()

,price,bedrooms,sqft_living,grade,sqft_lot
0,221900.0,3,1180,7,5650
1,538000.0,3,2570,7,7242
2,180000.0,2,770,6,10000
3,604000.0,4,1960,7,5000
4,510000.0,3,1680,8,8080


An initial correlation table can reveal those categories most correlated with price.  The top four correlations are, in descending order of correlation: sqft_living, grade, bathrooms, and bedrooms.

In [391]:
king.corr()

,price,bedrooms,sqft_living,grade,sqft_lot
price,1.000000,0.308787,0.701917,0.667951,0.089876
bedrooms,0.308787,1.000000,0.578212,0.356563,0.032471
sqft_living,0.701917,0.578212,1.000000,0.762779,0.173453
grade,0.667951,0.356563,0.762779,1.000000,0.114731
sqft_lot,0.089876,0.032471,0.173453,0.114731,1.000000


Exploratory visualization reveals that square footage of the lot seems to be heavily positively skewed. 
This fact confirmed below by the skew statistic

In [392]:
print('distribution skew:' ,king['sqft_lot'].skew())
print('distribution median:',king['sqft_lot'].median())
print('distribution standard deviation:', round(king['sqft_lot'].std()))

distribution skew: 13.072603567136046
distribution median: 7618.0
distribution standard deviation: 41413


The scatter plot of lot size to price of the lot from the from the exploratory visualization seems to indicate three natural divisions of the distribution.  The first corresponds to smaller, and more expensive urban lots.  The high price of these lots reflects the densly packed and expense of urban real estate.  The second division corresponds to less expensive and larger suburban lots.  The third division represents larger and less expensive rural real estate.  We bin and parse out into dummy variables this column.  

In [393]:
bins = [0,8000, 40000, 500000]

bin_names = ['urban', 'suburban', 'rural']

king['sqft_lot'] = pd.cut(king['sqft_lot'], bins, labels = bin_names)

lot_dummies = pd.get_dummies(king.sqft_lot).iloc[:,:2]

king = pd.concat([king, lot_dummies], axis = 1)

king.drop(['sqft_lot'], axis = 1, inplace = True)

Next, we define a column that appears, from exporatory analysis, to be relevant to house price.  Occupancy per square foot represents a rough estimate of space available to an occupant.  We calculate this value by dividing square foot of living space by the number of bedrooms

In [394]:
king['occupancy_per_sqft'] = round(king['sqft_living']/king['bedrooms'])

In [395]:
king.head()

,price,bedrooms,sqft_living,grade,urban,suburban,occupancy_per_sqft
0,221900.0,3,1180,7,1,0,393.0
1,538000.0,3,2570,7,1,0,857.0
2,180000.0,2,770,6,0,1,385.0
3,604000.0,4,1960,7,1,0,490.0
4,510000.0,3,1680,8,0,1,560.0


One of the most highly correlated variable to house price is the grade of the structure.  We break this variable into dummy variables. 

In [396]:
king_dummies = pd.get_dummies(king.grade).iloc[:,1:]
king_gradedum = pd.concat([king,king_dummies], axis = 1)

king_final = king_gradedum.drop(['grade'], axis = 1)

king_final.head()

,price,bedrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
0,221900.0,3,1180,1,0,393.0,0,0,0,1,0,0,0,0,0,0
1,538000.0,3,2570,1,0,857.0,0,0,0,1,0,0,0,0,0,0
2,180000.0,2,770,0,1,385.0,0,0,1,0,0,0,0,0,0,0
3,604000.0,4,1960,1,0,490.0,0,0,0,1,0,0,0,0,0,0
4,510000.0,3,1680,0,1,560.0,0,0,0,0,1,0,0,0,0,0


With the manipulations of the dataframe variables complete we break the dataframe into x and y variable for the regression analysis and then separate those divisions into training and testing batches.

In [397]:
king_y = pd.DataFrame(king_final.price)

king_x = king_final.drop(['price'], axis = 1)

king_train_x, king_test_x, king_train_y, king_test_y = train_test_split(
    king_x, king_y, test_size = .5, random_state = 5)
   

The head of the training set:

In [398]:
king_train_x.head()

,bedrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
8991,3,3490,1,0,1163.0,0,0,0,0,0,1,0,0,0,0
16827,5,2190,0,1,438.0,0,0,0,1,0,0,0,0,0,0
19166,3,2990,0,0,997.0,0,0,0,0,0,1,0,0,0,0
8072,4,2200,0,0,550.0,0,0,0,1,0,0,0,0,0,0
16264,2,1050,1,0,525.0,0,0,1,0,0,0,0,0,0,0


The first iteration of the regression analysis will use just the moderately correlated bathrooms and bedrooms variables.

In [428]:
king_train_x_a = king_train_x[[ 'bedrooms']]

In [429]:
king_train_x_a.head()

,bedrooms
8991,3
16827,5
19166,3
8072,4
16264,2


In [430]:

model = sm.OLS(king_train_y, king_train_x_a).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.720
Model:                            OLS   Adj. R-squared (uncentered):              0.720
Method:                 Least Squares   F-statistic:                          2.774e+04
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        10:30:18   Log-Likelihood:                     -1.5282e+05
No. Observations:               10798   AIC:                                  3.056e+05
Df Residuals:                   10797   BIC:                                  3.057e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
bedrooms     1.55e+05    930.567    166.566      0.000    1.53e+05    1.57e+05
==============================================================================
Omnibus:                     7998.144   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           391933.868
Skew:                           3.062   Prob(JB):                         0.00
Kurtosis:                      31.873   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R^2 is actually quite high at .773.  For the next iteration we use the metric we built by dividing living area by bedrooms.

In [431]:
king_train_x_b = king_train_x[[ 'occupancy_per_sqft']]

In [432]:
king_train_x_b.head()

,occupancy_per_sqft
8991,1163.0
16827,438.0
19166,997.0
8072,550.0
16264,525.0


In [433]:

model = sm.OLS(king_train_y, king_train_x_b).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.796
Model:                            OLS   Adj. R-squared (uncentered):              0.796
Method:                 Least Squares   F-statistic:                          4.204e+04
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:06:38   Log-Likelihood:                     -1.5112e+05
No. Observations:               10798   AIC:                                  3.022e+05
Df Residuals:                   10797   BIC:                                  3.022e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
occupancy_per_sqft   876.5943      4.275    205.033      0.000     868.214     884.975
==============================================================================
Omnibus:                     8299.577   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           413571.821
Skew:                           3.242   Prob(JB):                         0.00
Kurtosis:                      32.617   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A performance gain of about .025 R^2.  Next we incorporate lot size variable that we binned into urban, suburban, and rural categories and then divided into dummy categories.

In [434]:
king_train_x_c = king_train_x[['occupancy_per_sqft', 'urban', 'suburban']]

In [435]:
king_train_x_c.head()

,occupancy_per_sqft,urban,suburban
8991,1163.0,1,0
16827,438.0,0,1
19166,997.0,0,0
8072,550.0,0,0
16264,525.0,1,0


In [436]:

model = sm.OLS(king_train_y, king_train_x_c).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.796
Model:                            OLS   Adj. R-squared (uncentered):              0.796
Method:                 Least Squares   F-statistic:                          1.404e+04
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:07:03   Log-Likelihood:                     -1.5111e+05
No. Observations:               10798   AIC:                                  3.022e+05
Df Residuals:                   10795   BIC:                                  3.022e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
occupancy_per_sqft   888.9296      9.858     90.172      0.000     869.606     908.253
urban              -1.978e+04   6880.045     -2.876      0.004   -3.33e+04   -6297.519
suburban            3649.4634   7639.954      0.478      0.633   -1.13e+04    1.86e+04
==============================================================================
Omnibus:                     8157.766   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           396269.773
Skew:                           3.163   Prob(JB):                         0.00
Kurtosis:                      31.996   Cond. No.                     2.21e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.21e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

That varaible supplied a performance gain of about .01 R^2.  Next we incorporate the 'grade' variable that we divided into dummy categories.

In [437]:
king_train_x.head()

,bedrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
8991,3,3490,1,0,1163.0,0,0,0,0,0,1,0,0,0,0
16827,5,2190,0,1,438.0,0,0,0,1,0,0,0,0,0,0
19166,3,2990,0,0,997.0,0,0,0,0,0,1,0,0,0,0
8072,4,2200,0,0,550.0,0,0,0,1,0,0,0,0,0,0
16264,2,1050,1,0,525.0,0,0,1,0,0,0,0,0,0,0


In [438]:

model = sm.OLS(king_train_y, king_train_x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.876
Model:                            OLS   Adj. R-squared (uncentered):              0.876
Method:                 Least Squares   F-statistic:                              5069.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:07:24   Log-Likelihood:                     -1.4843e+05
No. Observations:               10798   AIC:                                  2.969e+05
Df Residuals:                   10783   BIC:                                  2.970e+05
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms           -1.898e+04   5280.157     -3.594      0.000   -2.93e+04   -8626.091
sqft_living          167.0150      8.947     18.667      0.000     149.477     184.553
urban               9.579e+04   9831.791      9.743      0.000    7.65e+04    1.15e+05
suburban            5.758e+04   9811.460      5.869      0.000    3.84e+04    7.68e+04
occupancy_per_sqft    -1.4950     28.636     -0.052      0.958     -57.627      54.637
4                   4.435e+04   5.76e+04      0.770      0.441   -6.86e+04    1.57e+05
5                   5.783e+04   2.86e+04      2.021      0.043    1739.361    1.14e+05
6                   8.344e+04   2.14e+04      3.908      0.000    4.16e+04    1.25e+05
7                   1.036e+05   2.16e+04      4.794      0.000    6.12e+04    1.46e+05
8                   1.699e+05   2.25e+04      7.553      0.000    1.26e+05    2.14e+05
9                   2.945e+05   2.38e+04     12.396      0.000    2.48e+05    3.41e+05
10                   4.95e+05   2.57e+04     19.264      0.000    4.45e+05    5.45e+05
11                  7.352e+05   2.99e+04     24.561      0.000    6.76e+05    7.94e+05
12                  1.221e+06   4.46e+04     27.370      0.000    1.13e+06    1.31e+06
13                   2.91e+06   9.15e+04     31.791      0.000    2.73e+06    3.09e+06
==============================================================================
Omnibus:                     5686.743   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            87430.421
Skew:                           2.173   Prob(JB):                         0.00
Kurtosis:                      16.245   Cond. No.                     1.02e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.02e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now for the moment of truth.  Validating the model with the testing batch.  The head of the testing x variable as follows:

This last regression provided the strongest R^2 at .87.

In [410]:
king_test_x.head()

,bedrooms,sqft_living,urban,suburban,occupancy_per_sqft,4,5,6,7,8,9,10,11,12,13
15393,4,1990,1,0,498.0,0,0,0,1,0,0,0,0,0,0
6035,4,6330,0,1,1582.0,0,0,0,0,0,0,0,0,0,1
12871,4,4270,0,1,1068.0,0,0,0,0,0,0,0,1,0,0
21099,4,1950,1,0,488.0,0,0,0,0,1,0,0,0,0,0
11629,2,1150,1,0,575.0,0,0,0,1,0,0,0,0,0,0


In [411]:
model = sm.OLS(king_test_y, king_test_x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     1127.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                        10:10:20   Log-Likelihood:            -1.4928e+05
No. Observations:               10799   AIC:                         2.986e+05
Df Residuals:                   10784   BIC:                         2.987e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms           -5.396e+04   7551.596     -7.146      0.000   -6.88e+04   -3.92e+04
sqft_living          238.7959     11.888     20.087      0.000     215.494     262.098
urban               9.911e+04    1.1e+04      9.028      0.000    7.76e+04    1.21e+05
suburban            6.438e+04    1.1e+04      5.859      0.000    4.28e+04    8.59e+04
occupancy_per_sqft  -132.5512     37.882     -3.499      0.000    -206.806     -58.296
4                   1.464e+05   8.09e+04      1.810      0.070   -1.22e+04    3.05e+05
5                   1.225e+05   3.34e+04      3.671      0.000    5.71e+04    1.88e+05
6                   1.323e+05   2.75e+04      4.810      0.000    7.84e+04    1.86e+05
7                   1.644e+05   2.82e+04      5.824      0.000    1.09e+05     2.2e+05
8                   2.116e+05   2.91e+04      7.278      0.000    1.55e+05    2.69e+05
9                   3.209e+05      3e+04     10.699      0.000    2.62e+05     3.8e+05
10                  5.058e+05   3.12e+04     16.205      0.000    4.45e+05    5.67e+05
11                  8.004e+05   3.43e+04     23.303      0.000    7.33e+05    8.68e+05
12                  1.334e+06   4.67e+04     28.564      0.000    1.24e+06    1.43e+06
13                  1.873e+06   1.05e+05     17.798      0.000    1.67e+06    2.08e+06
==============================================================================
Omnibus:                     7290.171   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           308530.177
Skew:                           2.695   Prob(JB):                         0.00
Kurtosis:                      28.625   Cond. No.                     1.11e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

R^2 = .610 These results were disapointing.   R^2 value of this model was significantly reduced from the previous regression.  The P value of the 'bathrooms' variable is decreased to irrelevance over past regressions.  Occupancy per square foot also saw a strange drop in P value.

In [412]:
#king_test_x_a = king_test_x.drop(['bathrooms'], axis = 1)

In [413]:
model = sm.OLS(king_test_y, king_test_x_a).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     1127.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                        10:10:21   Log-Likelihood:            -1.4928e+05
No. Observations:               10799   AIC:                         2.986e+05
Df Residuals:                   10784   BIC:                         2.987e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bedrooms           -5.396e+04   7551.596     -7.146      0.000   -6.88e+04   -3.92e+04
sqft_living          238.7959     11.888     20.087      0.000     215.494     262.098
urban               9.911e+04    1.1e+04      9.028      0.000    7.76e+04    1.21e+05
suburban            6.438e+04    1.1e+04      5.859      0.000    4.28e+04    8.59e+04
occupancy_per_sqft  -132.5512     37.882     -3.499      0.000    -206.806     -58.296
4                   1.464e+05   8.09e+04      1.810      0.070   -1.22e+04    3.05e+05
5                   1.225e+05   3.34e+04      3.671      0.000    5.71e+04    1.88e+05
6                   1.323e+05   2.75e+04      4.810      0.000    7.84e+04    1.86e+05
7                   1.644e+05   2.82e+04      5.824      0.000    1.09e+05     2.2e+05
8                   2.116e+05   2.91e+04      7.278      0.000    1.55e+05    2.69e+05
9                   3.209e+05      3e+04     10.699      0.000    2.62e+05     3.8e+05
10                  5.058e+05   3.12e+04     16.205      0.000    4.45e+05    5.67e+05
11                  8.004e+05   3.43e+04     23.303      0.000    7.33e+05    8.68e+05
12                  1.334e+06   4.67e+04     28.564      0.000    1.24e+06    1.43e+06
13                  1.873e+06   1.05e+05     17.798      0.000    1.67e+06    2.08e+06
==============================================================================
Omnibus:                     7290.171   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           308530.177
Skew:                           2.695   Prob(JB):                         0.00
Kurtosis:                      28.625   Cond. No.                     1.11e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""